# preprocess a compiled dataset used in Cano & Hahn (2024)

Cano & Hahn (2024) compiled FEMA, FSF, ACS Census, and PRISM Rainfall data. We will preprocess this and use it as a training/testing dataset for `CausalFlow`

In [1]:
import os
import numpy as np
from astropy.table import Table

In [2]:
fema = Table.read('/Users/chahah/data/noah/zipcode.fema.fsf.acs.rainfall.csv')

In [3]:
fema

reportedZipcode,yearOfLoss,monthOfLoss,state,longitude,latitude,crs_treat,communityRatingSystemDiscount,primaryResidence,policyCount,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,amountPaidOnTotalClaim,totalTotalInsuranceCoverage,zipcode,count_property,count_fema_sfha,pct_fema_sfha,count_fs_risk_2020_5,pct_fs_risk_2020_5,count_fs_risk_2050_5,pct_fs_risk_2050_5,count_fs_risk_2020_100,pct_fs_risk_2020_100,count_fs_risk_2050_100,pct_fs_risk_2050_100,count_fs_risk_2020_500,pct_fs_risk_2020_500,count_fs_risk_2050_500,pct_fs_risk_2050_500,pct_fs_fema_difference_2020,avg_risk_score_all,avg_risk_score_2_10,avg_risk_fsf_2020_100,avg_risk_fsf_2020_500,avg_risk_score_sfha,avg_risk_score_no_sfha,count_floodfactor1,count_floodfactor2,count_floodfactor3,count_floodfactor4,count_floodfactor5,count_floodfactor6,count_floodfactor7,count_floodfactor8,count_floodfactor9,count_floodfactor10,2018.population,2018.median_household_income,2018.renter_fraction,2018.educated_fraction,2018.white_fraction,2014.population,2014.median_household_income,2014.renter_fraction,2014.educated_fraction,2014.white_fraction,2010.population,2010.median_household_income,2010.renter_fraction,2010.educated_fraction,2010.white_fraction,AREA,mean_rainfall,sum_rainfall,median_rainfall,population,median_household_income,renter_fraction,educated_fraction,white_fraction,amountPaidOnTotalClaim_per_policy
int64,int64,int64,str2,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,int64,int64,int64,int64,float64,int64,float64,int64,float64,int64,float64,int64,float64,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,int64,int64,float64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1001,2005,10,MA,-72.6,42.1,0.0,11.0,0.6666666666666666,3.0,5958.55,0.0,455800,26400,5958.55,482200,1001,5343,499,9.3,574,10.7,673,12.6,1208,22.6,1228,23.0,1478,27.7,1530,28.6,13.3,2.82,7.34,8.34,7.51,9.65,2.11,3813,52,104,122,40,279,234,15,144,540,16064,66088,0.21358316733067728,0.3477663785443089,0.912101593625498,17423,56714,0.21735636801928485,0.27717140661029976,0.9168340699075934,17380,63682,0.18601841196777905,0.24933451641526175,0.9336593785960875,0.005208333333341666,321.2156677246094,963.6470031738281,312.2799987792969,16064.0,66088.0,0.21358316733067728,0.3477663785443089,0.912101593625498,1986.1833333333334
1001,2011,4,MA,-72.6,42.0,0.0,11.0,1.0,1.0,10545.77,1473.46,250000,100000,12019.23,350000,1001,5343,499,9.3,574,10.7,673,12.6,1208,22.6,1228,23.0,1478,27.7,1530,28.6,13.3,2.82,7.34,8.34,7.51,9.65,2.11,3813,52,104,122,40,279,234,15,144,540,16064,66088,0.21358316733067728,0.3477663785443089,0.912101593625498,17423,56714,0.21735636801928485,0.27717140661029976,0.9168340699075934,17380,63682,0.18601841196777905,0.24933451641526175,0.9336593785960875,0.005208333333341666,147.1486612955729,441.44598388671875,147.62399291992188,16064.0,66088.0,0.21358316733067728,0.3477663785443089,0.912101593625498,12019.23
1001,2011,8,MA,-72.6,42.1,0.0,11.0,1.0,2.0,9320.56,0.0,262000,5000,9320.56,267000,1001,5343,499,9.3,574,10.7,673,12.6,1208,22.6,1228,23.0,1478,27.7,1530,28.6,13.3,2.82,7.34,8.34,7.51,9.65,2.11,3813,52,104,122,40,279,234,15,144,540,16064,66088,0.21358316733067728,0.3477663785443089,0.912101593625498,17423,56714,0.21735636801928485,0.27717140661029976,0.9168340699075934,17380,63682,0.18601841196777905,0.24933451641526175,0.9336593785960875,0.005208333333341666,266.2306620279948,798.6919860839844,266.02899169921875,16064.0,66088.0,0.21358316733067728,0.3477663785443089,0.912101593625498,4660.28
1001,2014,8,MA,-72.6,42.1,0.0,11.0,1.0,1.0,590.26,0.0,250000,100000,590.26,350000,1001,5343,499,9.3,574,10.7,673,12.6,1208,22.6,1228,23.0,1478,27.7,1530,28.6,13.3,2.82,7.34,8.34,7.51,9.65,2.11,3

In [10]:
out = Table()
out['ZIPCODE'] = fema['reportedZipcode']
out['YEAROFLOSS'] = fema['yearOfLoss']
out['MONTHOFLOSS'] = fema['monthOfLoss']
out['longitude'] = fema['longitude']
out['latitude'] = fema['latitude']
out['CRS_TREAT'] = np.array(fema['crs_treat']).astype(bool)
out['CRS_SCORE'] = fema['communityRatingSystemDiscount']

out['paid_claim_per_policy'] = fema['amountPaidOnTotalClaim_per_policy']
out['flood_risk_avg_score'] = fema['avg_risk_score_all']
out['mean_monthly_rainfall'] = fema['mean_rainfall']
    out['median_household_income'] = fema['median_household_income']
out['population'] = fema['population'] 
out['renter_fraction'] = fema['renter_fraction']
out['educated_fraction'] = fema['educated_fraction']
out['white_fraction'] = fema['white_fraction']

# supplement with additional FSF scores

In [11]:
floodrisk = Table.read('/Users/chahah/data/noah/floody.fsf.floodrisk_forecast.hdf5')

In [12]:
out['flood_risk100'] = np.empty(len(out))

for zcode in np.unique(out['ZIPCODE']): 
    is_zip0 = (out['ZIPCODE'] == zcode)
    is_zip1 = (floodrisk['ZIPCODE'] == zcode)

    if np.sum(is_zip1) != 1: raise ValueError

    out['flood_risk100'][is_zip0] = floodrisk['risk100.2022'][is_zip1][0]

## update PRISM precipitation data 
NO NEED. THEY ARE VIRTUALLY IDENTICAL

In [8]:
rainfall = Table.read('/Users/chahah/data/noah/floody.prism.precip.hdf5')

In [24]:
for i in range(100): 
    is_zip = (rainfall['ZIPCODE'] == out['ZIPCODE'][i])
    if np.sum(is_zip) != 1: raise ValueError
    print(out['mean_monthly_rainfall'][i], rainfall['precip.%i' % out['YEAROFLOSS'][i]][:,out['MONTHOFLOSS'][i]-1][is_zip][0])

321.2156677246094 310.27200317382807
147.1486612955729 148.41099548339844
266.2306620279948 266.02899169921875
99.75433349609375 106.25199890136719
384.734375 386.0090026855469
193.8615016937256 194.60699462890625
386.77833048502606 383.1679992675781
230.09366353352863 229.02200317382812
327.1296641031901 329.8909912109375
403.3772888183594 398.92401123046875
305.2252851213728 309.4209899902344
331.80557686941967 329.28900146484375
108.81371307373047 108.27999877929688
117.56942749023438 117.03099822998047
158.39100646972656 158.39100646972656
397.6507568359375 396.7900085449219
304.16100311279297 304.135009765625
147.73739624023438 151.66299438476562
349.15740356445315 349.6860046386719
195.14620361328124 196.34800720214844
252.2451965332031 246.25599670410156
361.36549377441406 377.6409912109375
102.54450225830078 101.99400329589844
181.7042663574219 180.56100463867188
411.74549865722656 432.4519958496093
154.47550201416016 154.43099975585935
245.81550216674805 244.22799682617188
84.

In [25]:
out

ZIPCODE,YEAROFLOSS,MONTHOFLOSS,longitude,latitude,CRS_TREAT,CRS_SCORE,paid_claim_per_policy,flood_risk_avg_score,mean_monthly_rainfall,median_household_income,population,renter_fraction,educated_fraction,white_fraction,flood_risk100
int64,int64,int64,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1001,2005,10,-72.6,42.1,False,11.0,1986.1833333333334,2.82,321.2156677246094,66088.0,16064.0,0.21358316733067728,0.3477663785443089,0.912101593625498,22.4
1001,2011,4,-72.6,42.0,False,11.0,12019.23,2.82,147.1486612955729,66088.0,16064.0,0.21358316733067728,0.3477663785443089,0.912101593625498,22.4
1001,2011,8,-72.6,42.1,False,11.0,4660.28,2.82,266.2306620279948,66088.0,16064.0,0.21358316733067728,0.3477663785443089,0.912101593625498,22.4
1001,2014,8,-72.6,42.1,False,11.0,590.26,2.82,99.75433349609375,66088.0,16064.0,0.21358316733067728,0.3477663785443089,0.912101593625498,22.4
1005,2005,10,-72.1,42.4,False,11.0,5900.0,2.02,384.734375,80060.0,5166.0,0.06736353077816493,0.37786472889882616,0.9440572977158344,13.6
1005,2011,9,-72.1,42.4,False,11.0,440.41,2.02,193.8615016937256,80060.0,5166.0,0.06736353077816493,0.37786472889882616,0.9440572977158344,13.6
1010,2005,10,-72.25,42.1,False,11.0,15472.065,2.08,386.77833048502606,75577.0,3663.0,0.0990990990990991,0.4061482820976492,0.844935844935845,13.7
1010,2008,9,-72.3,42.1,False,11.0,5708.93,2.08,230.09366353352863,75577.0,3663.0,0.0990990990990991,0.4061482820976492,0.844935844935845,13.7
1010,2011,8,-72.3,42.1,False,11.0,4778.93,2.08,327.1296641031901,75577.0,3663.0,0.0990990990990991,0.4061482820976492,0.844935844935845,13.7


In [27]:
out.write('/Users/chahah/data/noah/floody.fema.compiled.hdf5', format='hdf5', overwrite=True)

/opt/anaconda3/envs/noah/lib/python3.8/site-packages/astropy/io/misc/hdf5.py:283: UserWarning: table path was not set via the path= argument; using default path __astropy_table__
  warnings.warn(
